# Cloud native geospatial file formats

These notes are adapted from the <a href="https://guide.cloudnativegeo.org/zarr/zarr-in-practice.html" target="_blank">Cloud-Optimized Geospatial Formats Guide</a>, which is published the a <a href="https://creativecommons.org/licenses/by/4.0/" target="_blank">CC by 4.0 license</a>.

A key feature of cloud native geospatial file formats is that they allow partial reads (over a network). This means that a large file (e.g. a large array based dataset of weather data or vector dataset of millions of geometries and associated attributes) is located in cloud storage and you can subset a portion of this data and read it into your local environment (e.g. to work with in a Python program).

Typically, cloud native geospatial file formats let you subset data by geographic location (e.g. a bounding box), time or variable names or column headings.

This notebook demonstrates how to implement partial reads of raster and vector data from cloud native geospatial files stored on the cloud.

In [ ]:
!pip install aiohttp dask matplotlib requests xarray zarr geopandas pyarrow fsspec mapclassify

## Zarr data




<a href="https://psl.noaa.gov/data/gridded/data.gpcp.html" target="_blank">NOAA global precipitation climate project (`gpcp`)</a> data is stored as zarr files in the cloud at Pangeo-forge's object storage. This is a relatively large file (>2GB) storing global precipitation measuremens for over 9000 days. As this is a cloud native geospatial file format, we can explore the structure of the file's data and retrieve subsets of the file's data without having to download the entire file.

The `xarray` output shows the dataset information retrieved *lazily*. Initially, it has only downloaded information about the dataset. Data is only downloaded when you request it for a computation and you can subset only the data that you require.

In [ ]:
import xarray as xr

store = "https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/gpcp-feedstock/gpcp.zarr"
ds = xr.open_dataset(store, engine="zarr", chunks={}, consolidated=True)
ds

When working with large files (i.e. files that would be too big to store on your computer / local storage, too big to load into memory, would take too long to download, or would be slow to interact with), lazy loading means you can quickly query and explore the structure without having to download the data. The ability to subset data from files in the cloud means you can download only the data that is relevant to you and avoid having to download large amounts of redundant data.

The example below demonstrates how to subset and download only the precipitation data from January 2020. The first day's worth of data is visualised on the map.

In [ ]:
ds.sel(time=slice('2020-01-01', '2020-01-31')).precip[0].plot(figsize=(24,12))

## GeoParquet

GeoParquet is a file format for storing vector geospatial data. It is based on the parquet format for storage of tabular data. Each row in a GeoParquet file corresponds to a spatial feature (e.g. points, lines and polygons).

GeoParquet files efficiently compress data, reducing storage and data transfer costs, and organise data within a file by column chunks and row groups. This means you can subset data by row group and column chunks. The data is spatially partitioned, this means you can spatially filter row groups of interest.

![](https://cloudnativegeo.org/images/20241209-geoparquetfile_hu2879898360095922016.png)

*Source: [Cloud Native Geospatial](https://cloudnativegeo.org/blog/2024/12/interview-with-kyle-barron-on-geoarrow-and-geoparquet-and-the-future-of-geospatial-data-analysis/)*

The below code snippets demonstrate reading vector data from a GeoParquet file in the cloud into a Python environment. The vector data represents field boundaries in California.

The first code snippet reads the entire dataset from the cloud into a GeoPandas GeoDataFrame. The second code snippet reads a subset of columns and spatial features from a bounding box near Davis.

In [ ]:
import fsspec
import geopandas as gpd
from fsspec.implementations.http import HTTPFileSystem
import time

filesystem = HTTPFileSystem()

start = time.time()
gdf = gpd.read_parquet("https://data.source.coop/fiboa/us-ca-scm/us_ca_scm.parquet", filesystem=filesystem)
gdf
print(f"read time: {time.time() - start}")
print(f"number of features: {gdf.shape[0]}")

In [ ]:
start = time.time()
gdf = gpd.read_parquet(
    "https://data.source.coop/fiboa/us-ca-scm/us_ca_scm.parquet",
    columns=["id", "crop:name", "geometry"],
    bbox=(-121.69410969790363, 38.50618205905283, -121.59848230208829, 38.5900803946528),
    filesystem=filesystem)
gdf
print(f"read time: {time.time() - start}")
print(f"number of features: {gdf.shape[0]}")
# gdf.explore()